Código - Simulação da Cinética Enzimática
===========================================================

**Autores:**  

João Gabriel Lima Marengo  
Joaquim Junior Ferola Fonseca  
Sophia Alves da Silva

---

## Descrição do Projeto

Este notebook é o projeto final da disciplina Prática em Ciência de Dados, cursada no primeiro semestre do Bacharelado em Ciência e Tecnologia da Ilum Escola de Ciência. O objetivo dessa aplicação é aplicar e aprofundar os conhecimentos adquiridos ao longo do semestre em um projeto prático, apresentado em um simpósio interno na própria instituição, com arguição dos professores responsáveis pela disciplina.

## Objetivo

O principal objetivo deste projeto é explorar o potencial das bibliotecas Plotly e Dash para a criação de uma visualização gráfica interativa. A cinética enzimática foi escolhida como exemplo prático para aplicar essas ferramentas, permitindo manipular parâmetros como concentração do substrato, constantes de ionização, energia de ativação, entre outros. A partir desses parâmetros, é gerado um  gráfico tridimensional da velocidade enzimática, em que os eixos x e y representam a temperatura e o pH , respectivamente, e o eixo z representa a velocidade da reação.

Essa abordagem facilita uma compreensão visual e dinâmica dos fatores que influenciam a atividade enzimática, além de demonstrar as capacidades das bibliotecas para projetos de ciência de dados interativos.

## Índice

1. [Introdução às Bibliotecas](#1-introdução-às-bibliotecas)
   - [1.1. Biblioteca Plotly](#11-biblioteca-plotly)
   - [1.2. Biblioteca Dash](#12-biblioteca-dash)
   - [1.3. Outras Bibliotecas Utilizadas](#13-outras-biblitoecas-utilizadas)
2. [Criação da Função Base](#2-criação-da-função-base)
3. [Implementação do Código](#3-implementação-do-código)
4. [Resultados](#4-resultados)
5. [Conclusão](#5-conclusão)
6. [Contribuições](#6-contribuições)
7. [Licença](#7-licença)

## 1. Introdução às Bibliotecas

Dado que o propósito central deste projeto é explorar o potencial das bibliotecas Plotly e Dash na construção de visualizações interativas, é essencial apresentar brevemente suas principais características e funções. A seguir, explicamos como essas ferramentas, juntamente com outras bibliotecas de suporte, foram aplicadas na criação de uma simulação dinâmica da cinética enzimática, permitindo ao usuário interagir com variáveis experimentais e observar seus efeitos em tempo real.

### 1.1. Biblioteca Plotly

[Plotly](https://plotly.com/python/) é uma biblioteca de visualização gráfica altamente interativa e compatível com notebooks, scripts Python e aplicações web. Ela permite a criação de gráficos em 2D e 3D com grande nível de customização e interatividade.

Neste projeto, usamos `plotly.graph_objects` para construir superfícies tridimensionais que representam como a velocidade da reação enzimática varia com temperatura e pH.

### 1.2. Biblioteca Dash

[Dash](https://dash.plotly.com/) é um framework desenvolvido pelos criadores do Plotly que permite construir interfaces web interativas em Python.

Com o Dash, podemos criar dashboards dinâmicos e interativos em que a visualização de dados é atualizada em tempo real a partir da interação do usuário com controles como sliders, inputs ou menus.

Este é o núcleo funcional da aplicação, servindo como base para a construção da interface web.

### 1.3. Outras Bibliotecas Utilizadas

* [NumPy](https://numpy.org/): biblioteca essencial para cálculos numéricos, especialmente na criação de matrizes e vetores usados para gerar os valores de temperatura e pH que alimentarão a superfície 3D.
* [Math](https://docs.python.org/pt-br/3/library/math.html): biblioteca padrão do Python utilizada para funções matemáticas como exponencial e potência.

In [ ]:
!pip install plotly dash numpy

In [ ]:
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import numpy as np
import math

In [ ]:
def velocidade_da_reacao(S, T, pH, pKa, pKb, Vmax_ref, Ea, T_ref, Km_ref, deltaH, T_opt, sigma):
    T_K = T + 273.15
    T_ref_K = T_ref + 273.15
    T_opt_K = T_opt + 273.15

    term1 = 1 / (1 + 10**(pKa - pH) + 10**(pH - pKb))
    exp1 = math.exp((-Ea * 1000 / 8.314) * (1 / T_K - 1 / T_ref_K))
    exp2 = math.exp((deltaH * 1000 / 8.314) * (1 / T_K - 1 / T_ref_K))
    fator_f = math.exp(-((T_K - T_opt_K) / sigma)**2)

    numerador = Vmax_ref * exp1 * fator_f * S
    denominador = Km_ref * exp2 + S

    return term1 * (numerador / denominador)

In [ ]:
app = Dash(__name__)

label_style = {
    'color': '#333333',
    'fontWeight': 'bold',
    'fontSize': '14px',
    'marginBottom': '6px',
    'display': 'block'
}

input_style = {
    'width': '100%',
    'padding': '8px 12px',
    'border': '1px solid #ccc',
    'borderRadius': '6px',
    'fontSize': '14px',
    'fontWeight': '400',
    'outline': 'none',
    'boxShadow': 'none',
    'transition': 'border-color 0.3s ease'
}

container_style = {
    'maxWidth': '1100px',
    'margin': '40px auto',
    'padding': '30px',
    'backgroundColor': '#f5f5f7',
    'borderRadius': '12px',
    'fontFamily': 'Trebuchet MS, sans-serif',
    'color': '#1d1d1f',
    'boxShadow': '0 4px 12px rgba(0,0,0,0.05)'
}

title_style = {
    'textAlign': 'center',
    'color': '#1d1d1f',
    'marginBottom': '40px',
    'fontWeight': '600',
    'fontSize': '28px'
}

app.layout = html.Div([
    html.H1("Simulação - Cinética Enzimática", style=title_style),

    html.Div([
        html.Div([
            html.Div([
                html.Label("Concentração do Substrato (mol/L)", style=label_style),
                dcc.Input(id='input_s', type='number', value=10, step=0.1, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Constante de Dissociação do Grupo Ácido (pKa)", style=label_style),
                dcc.Input(id='input_pka', type='number', value=6.5, step=0.1, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Constante de Dissociação do Grupo Básico (pKb)", style=label_style),
                dcc.Input(id='input_pkb', type='number', value=8, step=0.1, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Velocidade Máxima da Reação (mol/L.s)", style=label_style),
                dcc.Input(id='input_vmax_ref', type='number', value=32, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Energia de Ativação da Reação (kJ/mol)", style=label_style),
                dcc.Input(id='input_ea', type='number', value=50, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Temperatura de Referência (°C)", style=label_style),
                dcc.Input(id='input_tref', type='number', value=25, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Constante de Michaelis-Menten (mol/L)", style=label_style),
                dcc.Input(id='input_km_ref', type='number', value=0.4, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Variação de Entalpia (kJ/mol)", style=label_style),
                dcc.Input(id='input_deltah', type='number', value=-20, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Temperatura Ótima em Condições Ideais (°C)", style=label_style),
                dcc.Input(id='input_topt', type='number', value=37, style=input_style)
            ], style={'marginBottom': '10px'}),
            html.Div([
                html.Label("Sensibilidade Térmica (°C)", style=label_style),
                dcc.Input(id='input_sigma', type='number', value=10, style=input_style)
            ], style={'marginBottom': '10px'}),
        ], style={
            'flex': '0 0 380px',
            'paddingRight': '30px',
            'overflowY': 'auto',
            'maxHeight': '750px',
            'boxSizing': 'border-box'
        }),

        html.Div([
            dcc.Graph(id='graph-3d', style={'height': '700px'})
        ], style={'flex': '1'})
    ], style={'display': 'flex', 'flexDirection': 'row'})
], style=container_style)